<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/HAFS/hafs_download_animate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob as glob

from datetime import datetime, timedelta
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def fix_time_coordinates(fName):
    ds = xr.open_dataset(fName, decode_times=False)

    # Extract initial date string and convert to datetime object
    initial_date_str = ds.time.attrs['units'].split('since ')[1]
    initial_date = datetime.strptime(initial_date_str, '%Y-%m-%d %H:%M:%S')

    # Convert time offset values to datetime objects
    datetime_values = []
    for hour_offset in ds.time.values:
        datetime_values.append(initial_date + timedelta(hours=float(hour_offset)))

    # Assign the new datetime values to the 'time' coordinate
    ds['time'] = datetime_values

    return ds

## Inputs

In [ ]:
storm, forecast_start_date, cycle, which_hafs = ["08l", "20250925", "00", "hfsa"]

vars_to_extract = ['SSH']

url_base =\
f'https://noaa-nws-hafs-pds.s3.amazonaws.com/{which_hafs}/{forecast_start_date}/{cycle}/'

fName_pref = f'{storm}.{forecast_start_date}{cycle}.{which_hafs}.mom6.f'

fName_suff = '.nc'

In [ ]:
hrs = np.arange(0,123,3)
for hr in hrs:
  fName = fName_pref + str(hr).zfill(3) + fName_suff
  url = url_base + fName
  print(f'Working on\t{fName}')
  !wget -nc -q $url
  ds_fix_time = fix_time_coordinates(fName)
  ds_cull = ds_fix_time[vars_to_extract]

  fName_save = 'subset_' + fName
  ds_cull.to_netcdf(fName_save)
  print(f'Saved\n{fName_save}')

  ds_fix_time.close()
  ds_cull.close()
  !rm -f $fName

In [ ]:
ds = xr.open_mfdataset(glob.glob('subset_*'), combine='by_coords')

In [ ]:
!pip install cmocean
!pip install cartopy

!apt-get update && apt-get install -y ffmpeg
!pip install xmovie

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cmocean as cmo
from xmovie import Movie

In [ ]:
def pltVar(da, fig, tK, *args, **kwargs):
  cMin, cMax, cMap = [-0.6, 0.6, 'cmo.balance']

  ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree())

  # da is a DataArray
  im1=da.isel(time=tK).plot(ax=ax1, x='xh', y='yh', transform=ccrs.PlateCarree(), vmin=cMin, vmax=cMax, cmap=cMap, cbar_kwargs={'label': 'SSH [m]', 'pad':0.01, 'shrink':0.9, 'orientation': 'vertical'})

  # Get the current time from da_speed for comparison
  time_da = da.isel(time=tK).time.values

  # tStr should be from the dataarray's time coordinate
  tStr = da.isel(time=tK).time.dt.strftime('%Y/%m/%d %H UTC').values
  ax1.set_title(f'{tStr}')
  ax1.coastlines(resolution='10m', color='k')
  ax1.add_feature(cfeature.LAND, facecolor='lightgray')

  # Geographical region to plot/compare
  roi_lon = [-85, -60]
  roi_lat = [15, 30]
  ax1.set_extent([roi_lon[0], roi_lon[1], roi_lat[0], roi_lat[1]], crs=ccrs.PlateCarree())

  gl = ax1.gridlines(draw_labels=True, dms=True, linewidth=1, ls='--', color='k', alpha=0.5)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 8}
  gl.ylabel_style = {'size': 8}

  fig.tight_layout()
  #fig.savefig('test.png')

In [ ]:
fig = plt.figure(figsize=(12, 8))
tK=0
pltVar(ds['SSH'], fig, tK)

In [ ]:
save_fName_pref = 'ssh'
global_vmin, global_vmax = [-0.6, 0.6]

mov = Movie(ds['SSH'], pltVar, pixelwidth=2400, pixelheight=1600, dpi=300, input_check=False)

fName_animation = f'{save_fName_pref}_{storm}_{forecast_start_date}{cycle}.{which_hafs}.gif'
mov.save(fName_animation, progress=True, overwrite_existing=True, framerate=4, gif_framerate=4, gif_resolution_factor=1)
print(f"Saved animation to {fName_animation}")